In [1]:
import math
from tqdm import tqdm
import scvi
import torch
import numpy as np
import pytorch_lightning as pl
import os

import scanpy as sc

from pytorch_lightning.callbacks import TQDMProgressBar
from datamodules import FMDataset, fm_collate, CFMDataset, SCFMDataset, cfm_collate, StratifiedBatchSampler, ot_collate
from torch.utils.data import RandomSampler
from sc_etl_utils import *
from arch import *
from flow_utils import compute_conditional_flow
import json

import scvi
import torch
import numpy as np
import pytorch_lightning as pl
import os

import scanpy as sc

from torchcfm.conditional_flow_matching import *
import scanpy as sc
import hashlib
from llm import MAE

In [2]:
# load some data
adata = sc.read_h5ad('/orcd/archive/abugoot/001/Projects/dlesman/datasets/satija_IFNB_HVG_and_perturbed_genes_raw.h5ad')

In [3]:
gene_map = {'NT': 0} | {k: i for i, k in enumerate(adata.var.index)}
gene_unmap = {gene_map[k]: k for k in gene_map}
perts = adata.obs.gene.unique().map(gene_map)
adata.obs['pert_type'] = adata.obs.gene.map(gene_map)
pert_ids = np.array(adata.obs['pert_type'])
pert_mat = np.arange(pert_ids.max() + 1)[:, None]

In [4]:
cell_col = 'cell_type'
pert_col = 'pert_type'
control_pert, holdout_cells, holdout_perts = 0, ['HT29'], [gene_map['USP18']]

In [5]:
from sc_etl_utils import *
control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx = get_train_eval_idxs(
    adata, control_pert, holdout_cells, holdout_perts, cell_col=cell_col, pert_col=pert_col
)

_, _, cell_types = get_identity_features(
    adata, cell_col=cell_col, pert_col=pert_col, cell_type_features=False
)

adata.obsm["standard"] = adata.X
X = adata.obsm["standard"]
X = X.toarray()
X = X / X.sum(axis=1)[:, None]

control_train, pert_train, pert_ids_train, control_cell_types, pert_cell_types, control_eval, pert_eval, pert_ids_eval = get_train_eval(
    X, pert_ids, cell_types, control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx
)


Controls: 14582, Perturbations: 313080,  Eval: 880


In [6]:
batch_size = 512
base_learning_rate = 5e-5
weight_decay=0.0
total_epoch = 5000
warmup_epoch = 1

In [7]:
dset = SCFMDataset(
    control_train, pert_train, 
    pert_ids_train, pert_mat, 
    control_cell_types, pert_cell_types,
    batch_size=batch_size, size=X.shape[0]
)
dl = torch.utils.data.DataLoader(
    dset, collate_fn=ot_collate, 
    batch_sampler=StratifiedBatchSampler(
        RandomSampler(dset), batch_size=batch_size, drop_last=True, 
        probs=dset.probs, num_strata=dset.num_strata
    )
)

[0.09837271 0.2565357  0.2074058  0.17077354 0.08714468 0.17976757]


In [9]:
# model = MAE(X.shape[1])
model = torch.load(f"transformer_v4.1")
device = 'cuda'
# device = 'cpu'
model = model.to(device)

In [10]:
base_learning_rate = 5e-4
optim = torch.optim.AdamW(model.parameters(), lr=base_learning_rate, betas=(0.9, 0.95), weight_decay=weight_decay)
lr_func = lambda epoch: min((epoch + 1) / (warmup_epoch + 1e-5), 0.5 * (math.cos(epoch / total_epoch * math.pi) + 1))
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=lr_func, verbose=True)

Adjusting learning rate of group 0 to 5.0000e-04.


In [11]:
minibatch_size = 64

In [13]:
step_count = 0
optim.zero_grad()
pert_task = 0
for e in range(2):
    model.train()
    losses = {(0, 0): [], (0, 1): [], (1, 0): [], (1, 1): []}
    for (bcontrol, bpert, bpert_index) in (pbar := tqdm(iter(dl))):
        for i in range(batch_size // minibatch_size):
            control = bcontrol[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert = bpert[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert_index = bpert_index[(i * minibatch_size):((i + 1) * minibatch_size)]
            
            control_sparsity = (control > 0).float().to(device)
            active_weights = 10 * control_sparsity + 1
            control = control.float().to(device)
            pert = pert.float().to(device)
            step_count += 1
            mask_task = step_count % 2
            control_recon, sparsity_probs, _ = model(control, mask=mask_task)
            loss = torch.sum(active_weights * torch.abs(control_recon - control)) / minibatch_size
            loss += torch.sum(active_weights * torch.abs(control_sparsity - sparsity_probs)) / minibatch_size
            loss.backward()
            optim.step()
            optim.zero_grad()
            losses[(pert_task, mask_task)].append(loss.item())
        if step_count % 100 == 0:
            lr_scheduler.step()
        if pert_task == 0 and mask_task == 0:
            pbar.set_description(
                f"tv: {np.array(losses[(0, 0)])[-100:].mean():.3f}, mtv: {np.array(losses[(0, 1)])[-100:].mean():.3f}")
    
    avg_loss = sum(losses[(0, 0)]) / len(losses[(0, 0)])
    torch.save(model, f"transformer_v3.{e}")
    # writer.add_scalar('mae_loss', avg_loss, global_step=e)
    print(f'In epoch {e}, average traning loss is {avg_loss}.')

tv: 2375.500, mtv: 2534.008:   4%|█████▊                                                                                                                                              | 25/641 [03:06<1:16:42,  7.47s/it]

Adjusting learning rate of group 0 to 5.0000e-05.


tv: 2352.649, mtv: 2357.580:   8%|███████████▌                                                                                                                                        | 50/641 [06:13<1:13:37,  7.48s/it]

Adjusting learning rate of group 0 to 5.0000e-05.


tv: 2321.391, mtv: 2319.175:  12%|█████████████████▎                                                                                                                                  | 75/641 [09:20<1:10:36,  7.48s/it]

...

Adjusting learning rate of group 0 to 4.9988e-05.


tv: 1250.413, mtv: 1305.980:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 634/641 [1:19:11<00:52,  7.49s/it]

Adjusting learning rate of group 0 to 4.9987e-05.


tv: 1263.740, mtv: 1312.596: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 641/641 [1:20:03<00:00,  7.49s/it]

In epoch 1, average traning loss is 1292.4689273447402.


In [ ]:
step_count = 0
optim.zero_grad()
pert_task = 0
for e in range(10):
    model.train()
    losses = {(0, 0): [], (0, 1): [], (1, 0): [], (1, 1): []}
    for (bcontrol, bpert, bpert_index) in (pbar := tqdm(iter(dl))):
        for i in range(batch_size // minibatch_size):
            control = bcontrol[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert = bpert[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert_index = bpert_index[(i * minibatch_size):((i + 1) * minibatch_size)]
            active_weights = 10 * (control > 0).float().to(device) + 1
            control = control.float().to(device)
            pert = pert.float().to(device)
            step_count += 1
            mask_task = step_count % 2
            pert_task = (pert_task + mask_task) % 2
            if pert_task:
                pert_expr = pert[torch.arange(pert.size(0)), pert_index[:, 0], None]
                pert_recon, _, _ = model(control, pert_expr=pert_expr, pert_index=pert_index[:, 0], mask=mask_task)
                loss = torch.sum(active_weights * torch.abs(pert_recon - pert)) / minibatch_size
            else:
                control_recon, _, _ = model(control, mask=mask_task)
                loss = torch.sum(active_weights * torch.abs(control_recon - control)) / minibatch_size
            loss.backward()
            optim.step()
            optim.zero_grad()
            losses[(pert_task, mask_task)].append(loss.item())
            if step_count % 100 == 0:
                lr_scheduler.step()
            if pert_task == 0 and mask_task == 0:
                pbar.set_description(
                    f"tv: {np.array(losses[(0, 0)])[-100:].mean():.3f}, mtv: {np.array(losses[(0, 1)])[-100:].mean():.3f}, ptv: {np.array(losses[(1, 0)])[-100:].mean():.3f}, pmtv: {np.array(losses[(1, 1)])[-100:].mean():.3f}")
    
    avg_loss = sum(losses[(0, 0)]) / len(losses[(0, 0)])
    torch.save(model, f"transformer_v4.{e}")
    # writer.add_scalar('mae_loss', avg_loss, global_step=e)
    print(f'In epoch {e}, average traning loss is {avg_loss}.')

tv: 5.643, mtv: 8.000, ptv: 6.444, pmtv: 7.107:   4%|█████                                                                                                                            | 25/641 [03:14<1:19:56,  7.79s/it]

Adjusting learning rate of group 0 to 5.0000e-04.


tv: 5.461, mtv: 7.930, ptv: 6.438, pmtv: 7.193:   8%|██████████                                                                                                                       | 50/641 [06:29<1:16:48,  7.80s/it]

Adjusting learning rate of group 0 to 5.0000e-04.


tv: 5.242, mtv: 7.858, ptv: 6.420, pmtv: 7.264:  12%|███████████████                                                                                                                  | 75/641 [09:43<1:13:21,  7.78s/it]

...

tv: 3.280, mtv: 7.288, ptv: 6.127, pmtv: 7.077:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 634/641 [1:22:39<00:55,  7.89s/it]

Adjusting learning rate of group 0 to 4.9987e-04.


tv: 3.244, mtv: 7.230, ptv: 6.107, pmtv: 7.028: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 641/641 [1:23:35<00:00,  7.82s/it]


In epoch 1, average traning loss is 3.546174607671181.


tv: 3.249, mtv: 6.629, ptv: 6.050, pmtv: 6.706:   0%|▏                                                                                                                                 | 1/641 [00:07<1:24:30,  7.92s/it]

In [99]:
step_count = 0
optim.zero_grad()
pert_task = 0
for e in range(10):
    model.train()
    losses = {(0, 0): [], (0, 1): [], (1, 0): [], (1, 1): []}
    for (control, pert, pert_index) in (pbar := tqdm(iter(dl))):
        active_weights = 1 # 10 * (control > 0).float().to(device) + 1
        control = control.float().to(device)
        pert = pert.float().to(device)
        step_count += 1
        mask_task = step_count % 2
        pert_task = (pert_task + mask_task) % 2
        if pert_task:
            pert_expr = pert[torch.arange(pert.size(0)), pert_index[:, 0], None]
            pert_recon, _, _ = model(control, pert_expr=pert_expr, pert_index=pert_index[:, 0], mask=mask_task)
            loss = torch.sum(active_weights * torch.abs(pert_recon - pert)) / batch_size
        else:
            control_recon, _, _ = model(control, mask=mask_task)
            loss = torch.sum(active_weights * torch.abs(control_recon - control)) / batch_size
        loss.backward()
        optim.step()
        optim.zero_grad()
        if step_count % 100 == 0:
            lr_scheduler.step()
        losses[(pert_task, mask_task)].append(loss.item())
        if pert_task == 0 and mask_task == 0:
            pbar.set_description(
                f"tv: {np.array(losses[(0, 0)])[-100:].mean():.3f}, mtv: {np.array(losses[(0, 1)])[-100:].mean():.3f}, ptv: {np.array(losses[(1, 0)])[-100:].mean():.3f}, pmtv: {np.array(losses[(1, 1)])[-100:].mean():.3f}")
    
    avg_loss = sum(losses[(0, 0)]) / len(losses[(0, 0)])
    torch.save(model, f"transformer_v5.{e}")
    # writer.add_scalar('mae_loss', avg_loss, global_step=e)
    print(f'In epoch {e}, average traning loss is {avg_loss}.')

  0%|                                                                                                                                                                                            | 0/641 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 642.00 MiB. GPU 0 has a total capacty of 31.73 GiB of which 554.44 MiB is free. Including non-PyTorch memory, this process has 31.19 GiB memory in use. Of the allocated memory 30.39 GiB is allocated by PyTorch, and 431.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
cell_type_names = adata.obs[cell_col]
pert_type_names = adata.obs[pert_col]
# Save the predicted perturbation
for cell_type, pert_type in zip(holdout_cells, holdout_perts):
    control_eval = X[cell_type_names == cell_type]
    true_pert= X[(pert_type_names == pert_type) & (cell_type_names == cell_type)]
    # cheating right now, just trying to get a mvp together
    pert_expr = true_pert[torch.arange(true_pert.size(0)), pert_type, None]
    pred_pert, _, _ = model(
        control, pert_expr=pert_expr, pert_index=pert_type, mask=False
    )
    print(f"Saving {pert_type} predictions")
    np.savez(
        f"Satija_IFNB_HVG/llm/pred_{gene_unmap[pert_type]}_{cell_type}.npz", 
        pred_pert=pred_pert[:, hvg_idx].cpu().detach().numpy(), 
        true_pert=true_pert[:, hvg_idx], 
        control=control_eval[:, hvg_idx],
    )